# Isoration Forest 時系列異常検知
データを木構造で分類したとき，めったに出てこないサンプルの特徴は，木を深くしないと現れないという特徴を利用して外れ検知を行います。密度に基づく方法という意味では，kNNやLoFと似ていますが，空間を分割するため高次元でも計算量が爆発しないという利点があります。

### 必要なモジュールのimport

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

### セグメントの切り出し関数

In [ ]:
def segdata(lst, dim): #lst:１次元系列　dim:セグメントの幅
    emb = np.empty((0,dim), float)#0×dimの空配列
    for i in range(lst.size - dim + 1):#1つづつずらしながらセグメントをつくっている。最後のセグメントの開始点は lst.size-dim
        tmp = np.array(lst[i:i+dim])[::-1].reshape((1,-1)) #セグメントの切り出し，時系列反転，appendのための2次ベクトル化
        emb = np.append( emb, tmp, axis=0)
    return emb

データ読み込み，パラメータ設定
Keoghらの心電図のデータ http://www.cs.ucr.edu/~eamonn/discords/qtdbsel102.txt Keogh, E., Lin, J. and Fu, A.: HOT SAX : Efficiently Finding the Most Unusual Time Series Subsequence, in Proceedings of the Fifth IEEE International Conference on Data Mining, ICDM 05, pp.226-233.

In [ ]:
def getdata():
  !wget "www.dropbox.com/s/x3fmb9mxr4xkip3/qtdbsel102.txt" #ローカルにコピーしてくる
  LEN=3000  #分析区間

  SP=0         #学習用データの開始点
  AP=3000   #テスト用データの開始点　個のデータの場合 4250ポイント付近に異常がある
  data = np.loadtxt("qtdbsel102.txt",delimiter="\t")
  print("データ数:",data.shape[0],"  次元数:",data.shape[1])

  #元データは3次元の時系列，3次のデータ(indexとしては2)を指定して学習/テストデータに分割
  train_org = data[SP:SP+LEN, 2]      #学習用データとして 1～2999サンプル区間を使用
  test_org  = data[AP:AP+LEN, 2]    #テスト用データとして3000～5999サンプルを使用
  
  #x軸
  x=np.arange(SP,SP+LEN)

  return x, train_org, test_org

In [ ]:
x, train_org, test_org = getdata()

plt.plot(x,train_org)
plt.title("train")
plt.legend()
plt.show()
plt.plot(x,test_org)
plt.title("test with anomaly")
plt.legend()
plt.show()

WLEN=200 #セグメントのサイズ
train_seg = segdata(train_org, WLEN)
test_seg  = segdata(test_org, WLEN)

## isolation Forest（学習データ）

In [ ]:
outliers_fraction = 0.0001
rng = np.random.RandomState(123)
isof = IsolationForest(contamination=outliers_fraction,
                          max_samples="auto",
                          random_state=rng,
                          n_estimators=100)

isof.fit(train_seg) #学習
score_pred_train = isof.decision_function(train_seg) #判定スコア
pred = isof.predict(train_seg) #判定
outlier_rows = [i for i in range(len(pred)) if pred[i]==-1]

In [ ]:
for c in outlier_rows:
    plt.axvspan(c, c, color = "skyblue")
plt.plot(train_org)
plt.show()

## 未知データに対する評価

In [ ]:
score_pred_test = isof.decision_function(test_seg)
pred = isof.predict(test_seg)
outlier_rows = [i for i in range(len(pred)) if pred[i]==-1]
    

In [ ]:
for c in outlier_rows:
    plt.axvspan(c, c, color = "coral")
plt.plot(test_org)

plt.show()

In [ ]:
plt.plot(score_pred_train)
plt.plot(score_pred_test)
plt.show()